# Data Sources

In [1]:
import requests 
import pandas as pd
import json

In [2]:
with open('keys.json', 'r') as KeysFile:
    data = json.load(KeysFile)

aviationStackAPIKey = data["Aviation Stack API KEY"]

In [3]:
def getFutureFlights(departureIATA: str=None, arrivalIATA: str=None, date: str=None, airlineIATA: str=None, flightNumber: str=None):
    url = f"https://api.aviationstack.com/v1/flightsFuture?access_key={aviationStackAPIKey}"
    if not ((departureIATA is None) ^ (arrivalIATA is None)):
        return "Invalid Request"
    
    if departureIATA:
        queryString = {
            "iataCode": departureIATA,
            "type": "departure",
            "date": date,
            "airline_iata": airlineIATA,
            "flight_number":flightNumber
        }

    else:
        queryString = {
            "iataCode": arrivalIATA,
            "type": "arrival",
            "date": date,
            "airline_iata": airlineIATA,
            "flight_number":flightNumber
        }
    
    queryString = {key: value for key, value in queryString.items() if value is not None}
    response = requests.get(url, params=queryString)
    if response.status_code != 200:
        print(response)
        return None
    response = response.json()
    response = response['data']
    response = [flight for flight in response if "codeshared" not in flight.keys()]
    df = pd.DataFrame(columns=['Date', 'Weekday', 'DepartureIATA', 'DepartureTime', 'ArrivalIATA', 'ArrivalTime', 'Aircraft', 'AirlineIATA', 'FlightNumberIATA'])
    
    for flight in response:
        new_case = {'Date': date, 
                    'Weekday': flight['weekday'],
                    'DepartureIATA': flight['departure']['iataCode'],
                    'DepartureTime': flight['departure']['scheduledTime'],
                    'ArrivalIATA': flight['arrival']['iataCode'],
                    'ArrivalTime': flight['arrival']['scheduledTime'],
                    'Aircraft': flight['aircraft']['modelCode'],
                    'AirlineIATA': flight['airline']['iataCode'],
                    'FlightNumberIATA': flight['flight']['iataNumber']}
        df.loc[len(df)] = new_case

    return df

In [8]:
df = getFutureFlights(departureIATA="ATL", date="2024-11-29", airlineIATA="DL")

,Date,Weekday,DepartureIATA,DepartureTime,ArrivalIATA,ArrivalTime,Aircraft,AirlineIATA,FlightNumberIATA
0,2024-11-29,5,atl,20:31,elp,21:45,b739,dl,dl1713
1,2024-11-29,5,atl,21:34,grb,,a320,dl,dl2948
2,2024-11-29,5,atl,20:55,nce,12:00,b764,dl,dl24
3,2024-11-29,5,atl,21:18,yul,23:55,a319,dl,dl2416
4,2024-11-29,5,atl,21:20,jnb,19:00,a359,dl,dl200


In [30]:
flights_per_aircraft = df.groupby('ArrivalIATA').size()

# Optionally, sort the results in descending order
flights_per_aircraft.sort_values(ascending=False).head(50)

ArrivalIATA
mco    9
fll    8
lax    8
dca    8
lga    8
tpa    7
dtw    7
pbi    7
slc    6
clt    6
mci    6
sat    6
jax    6
rdu    6
pns    6
msp    6
mem    5
mia    5
mke    5
cmh    5
bwi    5
phl    5
bos    5
iah    5
pit    5
rsw    5
san    5
aus    5
sav    5
srq    5
dfw    5
hou    4
jfk    4
las    4
lex    4
msy    4
mdw    4
mex    4
okc    4
pdx    4
phx    4
sea    4
sju    4
vps    4
ecp    4
yyz    4
bna    4
bhm    4
bdl    4
cvg    3
dtype: int64